# KRX-Bench 합성 데이터셋 생성 with raw text 튜토리얼

- **litellm**은 다양한 LLM API를 OpenAI API로 통합하여 사용할 수 있는 라이브러리입니다.
- 본 튜토리얼에서는 OpenAI의 `gpt-4o-mini-2024-07-18` 모델을 활용하여 금융 관련 고품질 raw text 데이터셋 `alvanlii/finance-textbooks`을 QA Instruction 데이터셋으로 변환하는 방법에 대해 다룹니다.
- 데이터 생성 파이프라인:
 1. LLM에게 샘플링된 raw text를 입력으로 해서 질문 세트 생성
 2. 생성된 질문 세트에 대한 답변 생성

## 1. litellm 설치 및 환경 설정
- 필요 라이브러리: pandas, datasets, random, litellm, os

In [2]:
!pip install openai pandas tqdm

  Using cached openai-1.54.0-py3-none-any.whl.metadata (24 kB)
  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached tqdm-4.66.6-py3-none-any.whl.metadata (57 kB)
  Using cached anyio-4.6.2.post1-py3-none-any.whl.metadata (4.7 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.27.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached jiter-0.7.0-cp312-none-win_amd64.whl.metadata (5.3 kB)
  Using cached pydantic-2.9.2-py3-none-any.whl.metadata (149 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached numpy-2.1.3-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
import time
from tqdm import tqdm
import math
import pandas as pd
from openai import OpenAI
import os
import json
import matplotlib.pyplot as plt
# API 키 입력 (실행 시 입력하도록 설정)

api_key = input("API 키를 입력하세요: ")


In [4]:
# OpenAI 클라이언트 설정
client = OpenAI(api_key=api_key)

# 배치 크기 설정
BATCH_SIZE = 30  # 한 번에 처리할 요청 수
DELAY = 3  # 배치 간 대기 시간(초)

In [69]:
data = json.load(open('final.json', 'r', encoding='utf-8'))
ap = data.values()
data = pd.DataFrame(ap)

In [72]:
data['table'][0][0]

'<table>\n<colgroup>\n<col>\n<col>\n<col>\n<col>\n</colgroup>\n<thead>\n<tr>\n<th>매출유형</th>\n<th>품 목</th>\n<th>매출액</th>\n<th>비 중(%)</th>\n</tr>\n</thead>\n<tbody>\n<tr>\n<td>방송공연</td>\n<td>방송출연, 콘서트, 각종행사등</td>\n<td>5,375</td>\n<td>46.42</td>\n</tr>\n<tr>\n<td>공연,기획</td>\n<td>당사, 타사의 콘서트의 기획,진행등</td>\n<td>6,189</td>\n<td>53.45</td>\n</tr>\n<tr>\n<td>기 타</td>\n<td>영상제작등</td>\n<td>15</td>\n<td>0.13</td>\n</tr>\n<tr>\n<td>합  계</td>\n<td>11,579</td>\n<td>100</td>\n</tr>\n</tbody>\n</table>\n'

In [71]:
data['text'] = data['text'].apply(lambda x: x.replace('\xa0', ''))
for table in data['table']:
    for i in range(len(table)):
        table[i] = table[i].replace('\xa0', '')

In [73]:
with open('final2.json', 'w', encoding='utf-8') as f:
    json.dump(data.to_dict(), f, ensure_ascii=False, indent=4)


In [ ]:
# make batch

def make_batch(data, batch_size):
    batch = []
    for row in data.itertuples():
        if row.table!= []:
            for table in row.table:
                if len(table) < 2048:
                    
        batch.append(row.text)
        if len(batch) == batch_size:
            yield batch
            batch = []
            

In [ ]:
def batch_completion(messages_list, model="gpt-4o-mini-2024-07-18"):
    """
    배치로 OpenAI API 요청을 처리하는 함수
    """
    responses = []
    for messages in messages_list:
        try:
            response = client.chat.completions.create(
                model=model,
                messages=messages
            )
            responses.append(response)
        except Exception as e:
            print(f"Error in API call: {str(e)}")
            raise e
    return responses

def get_random_section(text, max_length):
    """
    텍스트에서 랜덤한 섹션을 추출하는 함수
    """
    import random


    if len(text) <= max_length:
        return text

    # 시작 위치를 랜덤하게 선택
    max_start = len(text) - max_length
    start = random.randint(0, max_start)

    # 문장이 중간에 잘리지 않도록 조정
    # 마침표나 줄바꿈을 찾아서 적절한 위치에서 자르기
    section = text[start:start + max_length]

    # 첫 문장이 중간에 잘렸다면 다음 마침표까지 이동
    first_period = section.find('.')
    if first_period != -1:
        section = section[first_period + 1:].strip()

    # 마지막 문장이 중간에 잘렸다면 마지막 마침표까지만 포함
    last_period = section.rfind('.')
    if last_period != -1:
        section = section[:last_period + 1].strip()

    return section

def process_in_batches(book_text, total_samples=10000):
    texts = []
    all_questions = []
    all_responses = []

    # 전체 배치 수 계산
    num_batches = math.ceil(total_samples / BATCH_SIZE)

    # 텍스트 샘플링
    print("텍스트 샘플링 중...")
    for i, sample in tqdm(enumerate(total_samples)):
        print(f"샘플링 중... {i + 1}/{total_samples}")
        texts.append(get_random_section(sample, 2048))

    # 배치 단위로 처리
    print("\n배치 처리 시작...")
    for batch in tqdm(range(num_batches)):
        start_idx = batch * BATCH_SIZE
        end_idx = min((batch + 1) * BATCH_SIZE, total_samples)

        try:
            # 1. 질문 생성
            batch_texts = texts[start_idx:end_idx]
            qrys = []
            for t in batch_texts:
                messages = [
                    {"role": "system",
                     "content": "Your job is creating multi-hop reasoning questions in fluent Korean. You will be given a part of a text. Make a question based on it. The question should require multiple steps of reasoning related to the text. Return the question only without any other text"},
                    {"role": "user",
                     "content": t}
                ]
                qrys.append(messages)

            print(f"\n배치 {batch + 1}/{num_batches}: 질문 생성 중...")
            question_responses = batch_completion(qrys)
            batch_questions = [r.choices[0].message.content for r in question_responses]

            # 2. 답변 생성
            qrys = []
            for q in batch_questions:
                messages = [
                    {"role": "system",
                     "content": "You are a skilled financial expert in Korea. Make a response for the question. DO NOT introduce yourself"},
                    {"role": "user",
                     "content": q}
                ]
                qrys.append(messages)

            print(f"배치 {batch + 1}/{num_batches}: 답변 생성 중...")
            answer_responses = batch_completion(qrys)
            batch_answers = [r.choices[0].message.content for r in answer_responses]

            # 결과 저장
            all_questions.extend(batch_questions)
            all_responses.extend(batch_answers)

            # 배치 중간 결과 저장
            temp_df = pd.DataFrame({
                'sampled_text': texts[:end_idx],
                'question': all_questions,
                'response': all_responses
            })
            temp_df.to_csv('qa_results_temp.csv', index=False, encoding="utf-8-sig")

            print(f"배치 {batch + 1} 완료. {DELAY}초 대기 중...")
            time.sleep(DELAY)

        except Exception as e:
            print(f"Error in batch {batch}: {str(e)}")
            # 오류 발생 시 현재까지의 결과 저장
            temp_df = pd.DataFrame({
                'sampled_text': texts[:end_idx],
                'question': all_questions,
                'response': all_responses
            })
            temp_df.to_csv('qa_results_error.csv', index=False, encoding="utf-8-sig")
            raise e

    # 최종 DataFrame 생성
    final_df = pd.DataFrame({
        'sampled_text': texts,
        'question': all_questions,
        'response': all_responses
    })

    return final_df


def main():
    try:
        # Google Drive 마운트
        # drive.mount('/content/drive')

        # 텍스트 파일 경로 설정 (예시)
        file_path = 'allone.txt'  # 실제 파일 경로로 수정하세요

        # 텍스트 파일 읽기
        print("텍스트 파일 로딩 중...")
        with open(file_path, 'r', encoding='utf-8-sig') as file:
            book_text = file.read()

        print("처리 시작...")
        df = process_in_batches(book_text, total_samples=5000)

        # 결과를 Google Drive에 저장
        output_path = 'qa_results_final_10000.csv'
        df.to_csv(output_path, index=False, encoding="utf-8-sig")
        print(f"처리 완료. 결과가 저장됨: {output_path}")

    except Exception as e:
        print(f"Process stopped due to error: {str(e)}")

# 실행
if __name__ == "__main__":
    main()

텍스트 파일 로딩 중...
처리 시작...
텍스트 샘플링 중...


100%|██████████| 5000/5000 [00:00<00:00, 125486.29it/s]



배치 처리 시작...


  0%|          | 0/167 [00:00<?, ?it/s]


배치 1/167: 질문 생성 중...
배치 1/167: 답변 생성 중...
배치 1 완료. 3초 대기 중...


...


배치 17/167: 질문 생성 중...
배치 17/167: 답변 생성 중...
배치 17 완료. 3초 대기 중...


 10%|█         | 17/167 [49:13<7:25:26, 178.18s/it]


배치 18/167: 질문 생성 중...
배치 18/167: 답변 생성 중...
배치 18 완료. 3초 대기 중...


 11%|█         | 18/167 [51:48<7:05:17, 171.26s/it]


배치 19/167: 질문 생성 중...
배치 19/167: 답변 생성 중...


### 끝!!!!!


## 2. 함수 선언

- 본 튜토리얼에서 사용하는 데이터셋인 `alvanlii/finance-textbooks`는 긴 길이의 텍스트 데이터로 구성되어 있기 때문에 적절한 길이의 데이터로 샘플링하는 작업이 필요합니다.
- `get_random_section`함수는 매우 긴 텍스트에서 무작위로 일부분을 샘플링하기 위한 함수로, 이 함수를 활용하면 긴 텍스트 데이터에서 적절한 길이의 텍스트 데이터를 샘플링해서 사용할 수 있습니다.

In [31]:
def get_random_section(text, max_length=1000):
    """텍스트에서 랜덤한 섹션을 추출하는 함수"""
    # 줄바꿈을 공백으로 대체하여 하나의 연속된 텍스트로 만듦
    text = ' '.join(text.split())

    if len(text) <= max_length:
        return text

    start = random.randint(0, len(text) - max_length)

    # 앞쪽으로 이동하여 마침표를 찾음
    while start > 0 and text[start] != '.':
        start -= 1
    start = start + 1 if start > 0 else 0

    section = text[start:start + max_length]

    # 마지막 문장이 완성되도록 조정
    last_period = section.rfind('.')
    if last_period != -1:
        section = section[:last_period + 1]

    return section.strip()

## 3. 데이터셋 로드 및 전처리

- `alvanlii/finance-textbooks` 데이터셋은 금융 관련 고품질 raw text 데이터셋으로 instruction 형태가 아닌 raw text로 구성되어 있습니다.
- 데이터의 길이가 굉장히 길기 때문에 사전의 정의한 `get_random_section` 함수로 샘플링을 진행한 뒤 활용합니다.

In [33]:
# TXT 파일 읽기
with open('allone.txt', 'r', encoding='utf-8') as f:
    book_text = f.read()


In [ ]:

# 데이터셋 샘플링
texts = []
for i in range(10):  # 원하는 샘플 수만큼 조정
    texts.append(get_random_section(book_text, 2048))

## 4. 합성 데이터셋 생성

합성 데이터셋 생성 파이프라인은 다음과 같습니다.
1. 샘플링된 raw text 데이터를 사용해서 질문 데이터셋을 생성합니다.
2. 생성된 질문 데이터셋에 대한 답변을 생성합니다.

In [21]:
# chat prompt 포맷팅
qrys = []
for t in texts:
    messages = [
    {"content":"Your job is creating multi-hop reasoning questions in fluent Korean. You will be given a part of a text. Make a question based on it. The question should require multiple steps of reasoning related to the text. Return the question only without any other text.","role":"system"},
    {"content": t, "role": "user"}]
    qrys.append(messages)

# 1. raw text 데이터를 활용한 질문 생성
responses = batch_completion(
    model="gpt-4o-mini-2024-07-18",
    messages=qrys
)
resps = [i.choices[0].message.content for i in responses]
total_prompt_tokens_for_q = sum([r.usage.prompt_tokens for r in responses])
total_completion_tokens_for_q = sum([r.usage.completion_tokens for r in responses])
df = pd.DataFrame({'sampled_text':texts,'question':resps})


/usr/local/lib/python3.10/dist-packages/pydantic/main.py:390: UserWarning: Pydantic serializer warnings:
  Expected `CompletionTokensDetails` but got `dict` with value `{'audio_tokens': None, 'reasoning_tokens': 0}` - serialized value may not be as expected
  Expected `PromptTokensDetails` but got `dict` with value `{'audio_tokens': None, 'cached_tokens': 0}` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


In [22]:
# 답변 생성용 prompt 포맷팅
qrys = []
for t in resps:
    messages = [
    {"content":"You are a skilled financial expert in Korea. Make a response for the question. DO NOT introduce yourself.","role":"system"},
    {"content": t, "role": "user"}]
    qrys.append(messages)

# 2. 생성된 질문에 대한 답변 생성
responses = batch_completion(
    model="gpt-4o-mini-2024-07-18",
    messages=qrys
)
resps = [i.choices[0].message.content for i in responses]
df['response'] = resps
total_prompt_tokens_for_a = sum([r.usage.prompt_tokens for r in responses])
total_completion_tokens_for_a = sum([r.usage.completion_tokens for r in responses])

## 5. 데이터셋 생성 비용 확인

In [23]:
print('total prompt tokens:', total_prompt_tokens_for_q + total_prompt_tokens_for_a)
print('prompt token costs: $', round((total_prompt_tokens_for_q + total_prompt_tokens_for_a) / 1000000 * 0.150, 6))
print('total completion tokens:', total_completion_tokens_for_q + total_completion_tokens_for_a)
print('completion token costs: $', round((total_completion_tokens_for_q + total_completion_tokens_for_a) / 1000000 * 0.600, 6))

total prompt tokens: 13576
prompt token costs: $ 0.002036
total completion tokens: 4681
completion token costs: $ 0.002809


## 6. 데이터셋 저장 및 확인

In [27]:
# CSV 파일 저장
df.to_csv("txt_result.csv", index=False, encoding="utf-8-sig")

# Excel 파일 저장
# df.to_excel("content/txt_result.xlsx")



In [ ]:
# HuggingFace Hub 업로드 - token에 개인 HuggingFace 토큰을 입력해주시면 됩니다.
result_df = Dataset.from_pandas(df)
result_df.push_to_hub("hf/dataset", token="HF_TOKEN")

df.head()

## 참고자료

- [alvanlii/finance-textbooks](https://huggingface.co/datasets/alvanlii/finance-textbooks)
- [litellm Docs](https://docs.litellm.ai/)
- [Cosmopedia GitHub](https://github.com/huggingface/cosmopedia)
- [Cosmopedia Blog](https://huggingface.co/blog/cosmopedia)
- [Textbooks Are All You Need](https://arxiv.org/pdf/2306.11644)
- [Learning to Generate Instruction Tuning Datasets for Zero-Shot Task Adaptation](https://arxiv.org/abs/2402.18334)

In [34]:
import time
from tqdm import tqdm  # 진행 상황을 보여주는 프로그래스 바
import math

# 배치 크기 설정
BATCH_SIZE = 25  # 한 번에 처리할 요청 수
DELAY = 3  # 배치 간 대기 시간(초)

def process_in_batches(total_samples=10000):
    texts = []
    all_questions = []
    all_responses = []

    # 전체 배치 수 계산
    num_batches = math.ceil(total_samples / BATCH_SIZE)

    # 텍스트 샘플링
    for i in range(total_samples):
        texts.append(get_random_section(book_text, 2048))

    # 배치 단위로 처리
    for batch in tqdm(range(num_batches)):
        start_idx = batch * BATCH_SIZE
        end_idx = min((batch + 1) * BATCH_SIZE, total_samples)

        try:
            # 1. 질문 생성
            batch_texts = texts[start_idx:end_idx]
            qrys = []
            for t in batch_texts:
                messages = [
                    {"content":"Your job is creating multi-hop reasoning questions in fluent Korean. You will be given a part of a text. Make a question based on it. The question should require multiple steps of reasoning related to the text. Return the question only without any other text",
                     "role":"system"},
                    {"content": t, "role": "user"}
                ]
                qrys.append(messages)

            question_responses = batch_completion(
                model="gpt-4o-mini-2024-07-18",
                messages=qrys
            )
            batch_questions = [r.choices[0].message.content for r in question_responses]

            # 2. 답변 생성
            qrys = []
            for q in batch_questions:
                messages = [
                    {"content":"You are a skilled financial expert in Korea. Make a response for the question. DO NOT introduce yourself",
                     "role":"system"},
                    {"content": q, "role": "user"}
                ]
                qrys.append(messages)

            answer_responses = batch_completion(
                model="gpt-4o-mini-2024-07-18",
                messages=qrys
            )
            batch_answers = [r.choices[0].message.content for r in answer_responses]

            # 결과 저장
            all_questions.extend(batch_questions)
            all_responses.extend(batch_answers)

            # 배치 중간 결과 저장
            temp_df = pd.DataFrame({
                'sampled_text': texts[:end_idx],
                'question': all_questions,
                'response': all_responses
            })
            temp_df.to_csv('qa_results_temp.csv', index=False, encoding="utf-8-sig")

            # 배치 간 대기
            time.sleep(DELAY)

        except Exception as e:
            print(f"Error in batch {batch}: {str(e)}")
            # 오류 발생 시 현재까지의 결과 저장
            temp_df = pd.DataFrame({
                'sampled_text': texts[:end_idx],
                'question': all_questions,
                'response': all_responses
            })
            temp_df.to_csv('qa_results_error.csv', index=False, encoding="utf-8-sig")
            raise e

    # 최종 DataFrame 생성
    final_df = pd.DataFrame({
        'sampled_text': texts,
        'question': all_questions,
        'response': all_responses
    })

    return final_df

# 실행
try:
    df = process_in_batches(total_samples=10000)
    df.to_csv('qa_results_final_10000.csv', index=False, encoding="utf-8-sig")
except Exception as e:
    print(f"Process stopped due to error: {str(e)}")

  0%|          | 0/200 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/pydantic/main.py:390: UserWarning: Pydantic serializer warnings:
  Expected `CompletionTokensDetails` but got `dict` with value `{'audio_tokens': None, 'reasoning_tokens': 0}` - serialized value may not be as expected
  Expected `PromptTokensDetails` but got `dict` with value `{'audio_tokens': None, 'cached_tokens': 1024}` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
  1%|          | 2/200 [00:31<51:43, 15.67s/it]/usr/local/lib/python3.10/dist-packages/pydantic/main.py:390: UserWarning: Pydantic serializer warnings:
  Expected `CompletionTokensDetails` but got `dict` with value `{'audio_tokens': None, 'reasoning_tokens': 0}` - serialized value may not be as expected
  Expected `PromptTokensDetails` but got `dict` with value `{'audio_tokens': None, 'cached_tokens': 1280}` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
  


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.I

  2%|▎         | 5/200 [01:21<52:49, 16.25s/it]

Error in batch 5: 'RateLimitError' object has no attribute 'choices'
Process stopped due to error: All arrays must be of the same length
